In [3]:
import sagemaker
import numpy as np
import pandas as pd
import boto3
from sklearn.model_selection import train_test_split

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\HARISH\AppData\Local\sagemaker\sagemaker\config.yaml


In [4]:
sess = sagemaker.Session()
region = sess.boto_session.region_name
sm_boto3 = boto3.client("sagemaker")
bucket = 'bucketcapstonecropwise'
print(f"Using bucket :{bucket}")

Using bucket :bucketcapstonecropwise


In [5]:
df = pd.read_csv("data\Optimizing Agriculture Production.csv")


In [6]:
x = df.drop('label',axis=1)
y = df['label']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0 , stratify=y)

In [8]:
trainX = pd.DataFrame(X_train)
trainX['label'] = y_train

testX = pd.DataFrame(X_test)
testX['label'] = y_test

In [11]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [12]:
sk_prefix = "sagemaker/cropwise/sklearncontainer"
trainpath = sess.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)
print(trainpath)
print(testpath)

s3://bucketcapstonecropwise/sagemaker/cropwise/sklearncontainer/train-V-1.csv
s3://bucketcapstonecropwise/sagemaker/cropwise/sklearncontainer/test-V-1.csv


In [13]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 3,n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Writing script.py


In [14]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::533267235904:role/service-role/AmazonSageMaker-ExecutionRole-20240605T150664",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [15]:
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-06-05-19-55-54-647


2024-06-05 19:56:02 Starting - Starting the training job...
2024-06-05 19:56:18 Starting - Preparing the instances for training...
2024-06-05 19:56:53 Downloading - Downloading input data...
2024-06-05 19:57:19 Downloading - Downloading the training image...
2024-06-05 19:58:10 Training - Training image download completed. Training in progress.
2024-06-05 19:58:10 Uploading - Uploading generated training model2024-06-05 19:58:04,964 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-06-05 19:58:04,967 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-05 19:58:05,009 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-06-05 19:58:05,253 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-05 19:58:05,265 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-05 19:58:05,277 sagemaker-training-toolkit INFO

In [ ]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)

In [ ]:
artifact

In [ ]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role="arn:aws:iam::566373416292:role/service-role/AmazonSageMaker-ExecutionRole-20230120T164209",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [ ]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)
endpoint_name

In [16]:
testX[features][0:2].values.tolist()


NameError: name 'features' is not defined

In [ ]:
testX[features][0:2].values.tolist()


In [ ]:
print(predictor.predict(testX[features][0:2].values.tolist()))


In [ ]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)